In [17]:
from PIL import Image
import numpy as np

# Carga la imagen con PIL
img = Image.open('ejemplo.jpg')

# Convierte la imagen a un array de numpy
img_array = np.array(img)


In [26]:
from PIL import Image
import numpy as np
import pycuda.autoinit
import pycuda.driver as drv
from pycuda.compiler import SourceModule

# Cargar la imagen
img = Image.open('ejemplo.jpg')
width, height = img.size

# Crear una máscara de color azul del mismo tamaño que la imagen original
mask = Image.new('RGB', (width, height), color='blue')

# Convertir la imagen y la máscara a arrays de numpy
img_np = np.array(img)
mask_np = np.array(mask)

# Crear el kernel de CUDA
mod = SourceModule("""
    __global__ void blend_images(unsigned char *img, unsigned char *mask, int width, int height)
    {
        int idx = threadIdx.x + blockIdx.x * blockDim.x;
        int idy = threadIdx.y + blockIdx.y * blockDim.y;
        if (idx < width && idy < height)
        {
            // Superponer la máscara en la imagen
            img[(idy * width + idx) * 3] = (img[(idy * width + idx) * 3] + mask[(idy * width + idx) * 3]) / 2;
            img[(idy * width + idx) * 3 + 1] = (img[(idy * width + idx) * 3 + 1] + mask[(idy * width + idx) * 3 + 1]) / 2;
            img[(idy * width + idx) * 3 + 2] = (img[(idy * width + idx) * 3 + 2] + mask[(idy * width + idx) * 3 + 2]) / 2;
        }
    }
""")

# Obtener la función del kernel
blend_images = mod.get_function("blend_images")

# Llamar al kernel de CUDA
blend_images(drv.InOut(img_np), drv.In(mask_np), np.int32(width), np.int32(height), block=(32, 32, 1), grid=(width//32, height//32))

# Convertir el array de numpy a una imagen de Pillow
img_blue = Image.fromarray(img_np)

# Guardar la imagen
img_blue.save('output.png')


In [27]:
from PIL import Image
import numpy as np

# Carga la imagen resultante con PIL
result_img = Image.open('output.png')

# Muestra la imagen resultante
result_img.show()


In [1]:
import cv2
import numpy as np
import pycuda.autoinit
import pycuda.driver as drv
from pycuda.compiler import SourceModule

# Crear el kernel de CUDA
mod = SourceModule("""
    __global__ void blend_frames(unsigned char *frame, int width, int height)
    {
        int idx = threadIdx.x + blockIdx.x * blockDim.x;
        int idy = threadIdx.y + blockIdx.y * blockDim.y;
        if (idx < width && idy < height)
        {
            // Cambiar los canales rojo y verde a 0
            frame[(idy * width + idx) * 3] = 0;
            frame[(idy * width + idx) * 3 + 1] = 0;
        }
    }
""")

# Obtener la función del kernel
blend_frames = mod.get_function("blend_frames")

# Abrir la cámara
cap = cv2.VideoCapture(0)

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret:
        # Convertir el fotograma a un array de numpy
        frame_np = np.array(frame)

        # Llamar al kernel de CUDA
        blend_frames(drv.InOut(frame_np), np.int32(frame_np.shape[1]), np.int32(frame_np.shape[0]), block=(32, 32, 1), grid=(frame_np.shape[1]//32, frame_np.shape[0]//32))

        # Mostrar el fotograma
        cv2.imshow('Video', frame_np)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

# Liberar los recursos
cap.release()
cv2.destroyAllWindows()


efecto de pintura al oleo

In [9]:
from PIL import Image
import numpy as np

def load_image(image_path):
    image = Image.open(image_path)
    return np.array(image)

image_path = 'rostro.jpg'
image = load_image(image_path)


In [10]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

# Kernel CUDA para efecto de pintura al óleo
oil_paint_kernel = """
__global__ void oil_paint(unsigned char *input, unsigned char *output, int width, int height, int radius, int levels)
{
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    if (x >= width || y >= height)
        return;

    int histogram[256] = {0};
    int intensity_sum[256][3] = {0};

    for (int ky = -radius; ky <= radius; ky++)
    {
        for (int kx = -radius; kx <= radius; kx++)
        {
            int px = min(max(x + kx, 0), width - 1);
            int py = min(max(y + ky, 0), height - 1);
            int idx = (py * width + px) * 3;

            int r = input[idx];
            int g = input[idx + 1];
            int b = input[idx + 2];
            int intensity = (r + g + b) / 3;

            histogram[intensity]++;
            intensity_sum[intensity][0] += r;
            intensity_sum[intensity][1] += g;
            intensity_sum[intensity][2] += b;
        }
    }

    int max_intensity = 0;
    for (int i = 1; i < 256; i++)
    {
        if (histogram[i] > histogram[max_intensity])
        {
            max_intensity = i;
        }
    }

    int r = intensity_sum[max_intensity][0] / histogram[max_intensity];
    int g = intensity_sum[max_intensity][1] / histogram[max_intensity];
    int b = intensity_sum[max_intensity][2] / histogram[max_intensity];

    int output_idx = (y * width + x) * 3;
    output[output_idx] = r;
    output[output_idx + 1] = g;
    output[output_idx + 2] = b;
}
"""

mod = SourceModule(oil_paint_kernel)
oil_paint = mod.get_function("oil_paint")



In [11]:
def apply_oil_painting_effect(image, radius=3, levels=256):
    height, width, channels = image.shape

    if channels != 3:
        raise ValueError("La imagen debe tener 3 canales de color (RGB).")

    # Asegurarse de que los datos estén en formato uint8
    image = image.astype(np.uint8)

    # Asignar memoria en la GPU
    input_image_gpu = cuda.mem_alloc(image.nbytes)
    output_image_gpu = cuda.mem_alloc(image.nbytes)

    # Copiar datos a la GPU
    cuda.memcpy_htod(input_image_gpu, image)

    # Ejecutar kernel
    block_size = (16, 16, 1)
    grid_size = (int(np.ceil(width / block_size[0])), int(np.ceil(height / block_size[1])), 1)

    oil_paint(input_image_gpu, output_image_gpu, np.int32(width), np.int32(height), np.int32(radius), np.int32(levels), block=block_size, grid=grid_size)

    # Crear un array vacío para almacenar la imagen procesada
    output_image = np.empty_like(image)

    # Copiar datos de vuelta a la CPU
    cuda.memcpy_dtoh(output_image, output_image_gpu)

    return output_image

output_image = apply_oil_painting_effect(image)


In [12]:
def save_image(image, output_path):
    output_image = Image.fromarray(image)
    output_image.save(output_path)

output_path = 'imagen_pintura_oleo.jpg'
save_image(output_image, output_path)


filtro vintage 

In [16]:
import numpy as np
from PIL import Image
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

# Cargar la imagen
def load_image(image_path):
    image = Image.open(image_path)
    image = image.convert('RGB')
    image_np = np.asarray(image, dtype=np.float32)
    return image_np

# Guardar la imagen
def save_image(image_np, output_path):
    image_np = np.clip(image_np, 0, 255).astype(np.uint8)
    image = Image.fromarray(image_np)
    image.save(output_path)

# Código CUDA para el efecto vintage
kernel_code = """
__global__ void vintage_filter(float *image, int width, int height)
{
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;
    
    if (x < width && y < height) {
        int idx = (y * width + x) * 3;
        
        float r = image[idx];
        float g = image[idx + 1];
        float b = image[idx + 2];
        
        // Convertir a escala de grises
        float gray = 0.3f * r + 0.59f * g + 0.11f * b;

        // Aplicar tintes vintage
        image[idx] = gray + 35.0f;       // Red
        image[idx + 1] = gray + 20.0f;   // Green
        image[idx + 2] = gray - 20.0f;   // Blue
    }
}
"""

# Configuración del kernel y ejecución
def apply_vintage_filter(image_np):
    height, width, _ = image_np.shape
    image_size = width * height * 3

    # Asignar memoria en la GPU
    image_gpu = cuda.mem_alloc(image_np.nbytes)
    
    # Copiar datos a la GPU
    cuda.memcpy_htod(image_gpu, image_np)

    # Compilar y ejecutar el kernel
    mod = SourceModule(kernel_code)
    vintage_filter = mod.get_function("vintage_filter")
    
    # Configuración del bloque e hilo
    block_size = (16, 16, 1)
    grid_size = (int(np.ceil(width / 16)), int(np.ceil(height / 16)), 1)
    
    vintage_filter(image_gpu, np.int32(width), np.int32(height), block=block_size, grid=grid_size)
    
    # Copiar el resultado de vuelta a la CPU
    cuda.memcpy_dtoh(image_np, image_gpu)
    
    return image_np

if __name__ == "__main__":
    # Ruta de la imagen de entrada
    image_path = "ejemplo.jpg"
    
    # Cargar la imagen
    image_np = load_image(image_path)
    
    # Aplicar el filtro vintage
    image_np_vintage = apply_vintage_filter(image_np)
    
    # Guardar la imagen procesada
    output_path = "imagen_vintage.jpg"
    save_image(image_np_vintage, output_path)
    
    print(f"Imagen guardada en: {output_path}")


Imagen guardada en: imagen_vintage.jpg


In [28]:
import cv2
import numpy as np
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

# Cargar el kernel de CUDA para el efecto vintage
kernel_code = """
__global__ void vintage_filter(unsigned char *frame, int width, int height, int channels)
{
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;
    
    if (x < width && y < height) {
        int idx = (y * width + x) * channels;
        
        float r = frame[idx];
        float g = frame[idx + 1];
        float b = frame[idx + 2];
        
        // Convertir a escala de grises
        float gray = 0.3f * r + 0.59f * g + 0.11f * b;

        // Aplicar tintes vintage
        frame[idx] = (unsigned char)(gray + 40.0f);     // Red
        frame[idx + 1] = (unsigned char)(gray + 25.0f); // Green
        frame[idx + 2] = (unsigned char)(gray + 10.0f); // Blue
    }
}
"""

# Configuración del kernel y ejecución
def apply_vintage_filter(frame):
    height, width, channels = frame.shape
    frame_size = width * height * channels

    # Asignar memoria en la GPU
    frame_gpu = cuda.mem_alloc(frame.nbytes)
    
    # Copiar datos a la GPU
    cuda.memcpy_htod(frame_gpu, frame)

    # Compilar y ejecutar el kernel
    mod = SourceModule(kernel_code)
    vintage_filter = mod.get_function("vintage_filter")
    
    # Configuración del bloque e hilo
    block_size = (32, 8, 1)  # Ajustar el tamaño del bloque
    grid_size = (int(np.ceil(width / block_size[0])), int(np.ceil(height / block_size[1])), 1)
    
    vintage_filter(frame_gpu, np.int32(width), np.int32(height), np.int32(channels), block=block_size, grid=grid_size)
    
    # Copiar el resultado de vuelta a la CPU
    cuda.memcpy_dtoh(frame, frame_gpu)
    
    return frame

if __name__ == "__main__":
    # Inicializar la cámara
    cap = cv2.VideoCapture(0)
    
    while True:
        ret, frame = cap.read()
        
        if not ret:
            break
        
        # Aplicar el filtro vintage
        frame_vintage = apply_vintage_filter(frame)
        
        # Mostrar el video procesado
        cv2.imshow("Vintage Video", frame_vintage)
        
        # Salir si se presiona la tecla 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Liberar la cámara y cerrar todas las ventanas
    cap.release()
    cv2.destroyAllWindows()


efecto pintura al oleo usando video

In [25]:
import cv2
import numpy as np
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

# Kernel CUDA para efecto de pintura al óleo
oil_paint_kernel = """
__global__ void oil_paint(unsigned char *input, unsigned char *output, int width, int height, int radius, int levels)
{
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    if (x >= width || y >= height)
        return;

    int histogram[256] = {0};
    int intensity_sum[256][3] = {0};

    for (int ky = -radius; ky <= radius; ky++)
    {
        for (int kx = -radius; kx <= radius; kx++)
        {
            int px = min(max(x + kx, 0), width - 1);
            int py = min(max(y + ky, 0), height - 1);
            int idx = (py * width + px) * 3;

            int r = input[idx];
            int g = input[idx + 1];
            int b = input[idx + 2];
            int intensity = (r + g + b) / 3;

            histogram[intensity]++;
            intensity_sum[intensity][0] += r;
            intensity_sum[intensity][1] += g;
            intensity_sum[intensity][2] += b;
        }
    }

    int max_intensity = 0;
    for (int i = 1; i < 256; i++)
    {
        if (histogram[i] > histogram[max_intensity])
        {
            max_intensity = i;
        }
    }

    int r = intensity_sum[max_intensity][0] / histogram[max_intensity];
    int g = intensity_sum[max_intensity][1] / histogram[max_intensity];
    int b = intensity_sum[max_intensity][2] / histogram[max_intensity];

    int output_idx = (y * width + x) * 3;
    output[output_idx] = r;
    output[output_idx + 1] = g;
    output[output_idx + 2] = b;
}
"""

# Compilar el kernel
mod = SourceModule(oil_paint_kernel)
oil_paint = mod.get_function("oil_paint")

def apply_oil_painting_effect(frame, radius=3, levels=256):
    height, width, channels = frame.shape

    # Asegurarse de que los datos estén en formato uint8
    frame = frame.astype(np.uint8)

    # Asignar memoria en la GPU
    input_frame_gpu = cuda.mem_alloc(frame.nbytes)
    output_frame_gpu = cuda.mem_alloc(frame.nbytes)

    # Copiar datos a la GPU
    cuda.memcpy_htod(input_frame_gpu, frame)

    # Ejecutar kernel
    block_size = (16, 16, 1)
    grid_size = (int(np.ceil(width / block_size[0])), int(np.ceil(height / block_size[1])), 1)

    oil_paint(input_frame_gpu, output_frame_gpu, np.int32(width), np.int32(height), np.int32(radius), np.int32(levels), block=block_size, grid=grid_size)

    # Crear un array vacío para almacenar el frame procesado
    output_frame = np.empty_like(frame)

    # Copiar datos de vuelta a la CPU
    cuda.memcpy_dtoh(output_frame, output_frame_gpu)

    return output_frame

# Capturar el video desde la cámara
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()

    if not ret:
        break

    # Aplicar el filtro de pintura al óleo al frame
    frame_painted = apply_oil_painting_effect(frame)

    # Mostrar el video procesado
    cv2.imshow("Oil Paint Video", frame_painted)

    # Salir si se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar la cámara y cerrar todas las ventanas
cap.release()
cv2.destroyAllWindows()


In [2]:
import numpy as np
import pycuda.autoinit
import pycuda.driver as cuda
from pycuda.compiler import SourceModule
from PIL import Image

# Kernel para dibujar círculos y mantener el área dentro del círculo en color, y el área fuera en blanco y negro
kernel_code = """
__global__ void draw_circle_with_background(unsigned char *img, int width, int height, int cx, int cy, int radius) {
    int x = threadIdx.x + blockIdx.x * blockDim.x;
    int y = threadIdx.y + blockIdx.y * blockDim.y;
    if (x < width && y < height) {
        int dx = x - cx;
        int dy = y - cy;
        float distance = sqrtf(dx*dx + dy*dy);
        if (distance <= radius) {
            // Mantener el color dentro del círculo
            // No hacer nada, simplemente mantener el color original
        } else {
            // Convertir el área fuera del círculo a blanco y negro
            unsigned char gray_value = (unsigned char)(0.299*img[(y * width + x) * 3] + 0.587*img[(y * width + x) * 3 + 1] + 0.114*img[(y * width + x) * 3 + 2]);
            img[(y * width + x) * 3] = gray_value;
            img[(y * width + x) * 3 + 1] = gray_value;
            img[(y * width + x) * 3 + 2] = gray_value;
        }
    }
}
"""

def apply_circle_with_background_filter(image, radius, cx, cy):
    height, width, _ = image.shape

    # Copiar la imagen a la memoria de la GPU
    img_gpu = cuda.mem_alloc(image.nbytes)
    cuda.memcpy_htod(img_gpu, image)

    # Compilar el kernel
    mod = SourceModule(kernel_code)
    draw_circle_with_background_kernel = mod.get_function("draw_circle_with_background")

    # Definir las dimensiones del bloque y de la grilla
    block = (16, 16, 1)
    grid = ((width + block[0] - 1) // block[0], (height + block[1] - 1) // block[1])

    # Llamar al kernel
    draw_circle_with_background_kernel(img_gpu, np.int32(width), np.int32(height), np.int32(cx), np.int32(cy), np.int32(radius), block=block, grid=grid)

    # Copiar la imagen de vuelta a la CPU
    result = np.empty_like(image)
    cuda.memcpy_dtoh(result, img_gpu)

    # Liberar la memoria de la GPU
    img_gpu.free()

    return result

# Cargar una imagen desde un archivo usando Pillow
image_path = 'rostro.jpg'  # Reemplaza con la ruta a tu imagen
image_pil = Image.open(image_path).convert("RGB")

# Convertir la imagen de Pillow a un array de NumPy
image_np = np.array(image_pil)

# Obtener dimensiones de la imagen
height, width, _ = image_np.shape

# Definir el radio del círculo
radius = 100

# Definir el centro del círculo en el centro de la imagen
cx, cy = width // 2, height // 2

# Aplicar el filtro de círculos con fondo en blanco y negro
filtered_image_np = apply_circle_with_background_filter(image_np, radius, cx, cy)

# Convertir el array de NumPy de vuelta a una imagen de Pillow
filtered_image_pil = Image.fromarray(filtered_image_np)

# Guardar la imagen usando Pillow
filtered_image_pil.save('circulo.jpg')  # Especifica la ruta donde deseas guardar la imagen


In [11]:
import numpy as np
import cv2
import pycuda.autoinit
import pycuda.driver as cuda
from pycuda.compiler import SourceModule

def overlay_texture(image, texture):
    # Convertir la imagen y la textura a arrays numpy y asegurar que sean float32
    image = image.astype(np.float32)
    texture = texture.astype(np.float32)

    # Obtener dimensiones de la imagen y la textura
    height, width, channels = image.shape
    texture_height, texture_width, texture_channels = texture.shape

    # Redimensionar la textura si es más grande que la imagen
    if texture_height > height or texture_width > width:
        texture = cv2.resize(texture, (width, height))

    # Definir el kernel para superponer la textura
    overlay_kernel = SourceModule("""
        __global__ void overlay_texture(float *image, float *texture, int width, int height, int texture_width, int texture_height, int channels) {
            int x = threadIdx.x + blockIdx.x * blockDim.x;
            int y = threadIdx.y + blockIdx.y * blockDim.y;

            if (x < width && y < height) {
                int image_index = (y * width + x) * channels;
                int texture_index = (y * texture_width + x) * channels;
                for (int c = 0; c < channels; ++c) {
                    image[image_index + c] = fminf(image[image_index + c] + texture[texture_index + c], 255.0f);
                }
            }
        }
    """)

    # Obtener la función del kernel
    overlay_func = overlay_kernel.get_function("overlay_texture")

    # Transferir la imagen y la textura a la GPU
    image_gpu = cuda.mem_alloc(image.nbytes)
    texture_gpu = cuda.mem_alloc(texture.nbytes)
    cuda.memcpy_htod(image_gpu, image)
    cuda.memcpy_htod(texture_gpu, texture)

    # Definir las dimensiones del bloque y de la cuadrícula
    block = (16, 16, 1)
    grid = ((width + block[0] - 1) // block[0], (height + block[1] - 1) // block[1], 1)

    # Ejecutar el kernel en la GPU
    overlay_func(image_gpu, texture_gpu, np.int32(width), np.int32(height), np.int32(texture_width), np.int32(texture_height),
                 np.int32(channels), block=block, grid=grid)

    # Transferir la imagen de vuelta a la CPU
    cuda.memcpy_dtoh(image, image_gpu)

    # Convertir la imagen de vuelta a tipo uint8
    image = np.clip(image, 0, 255).astype(np.uint8)

    return image

# Cargar la imagen original
original_image = cv2.imread("rostro.jpg", cv2.IMREAD_COLOR)

# Cargar la textura a superponer
texture = cv2.imread("red-ius-dob-bosco.jpg", cv2.IMREAD_COLOR)

# Aplicar el filtro de superposición de texturas
result_image = overlay_texture(original_image, texture)

# Guardar la imagen resultante
cv2.imwrite("result_image.jpg", result_image)

# Mostrar la imagen resultante
cv2.imshow("Result Image", result_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [2]:
import numpy as np
import cv2
import pycuda.autoinit
import pycuda.driver as cuda
from pycuda.compiler import SourceModule

def overlay_texture(image, texture, intensity=0.5):
    # Convertir la imagen y la textura a arrays numpy y asegurar que sean float32
    image = image.astype(np.float32)
    texture = texture.astype(np.float32)

    # Obtener dimensiones de la imagen y la textura
    height, width, channels = image.shape
    texture_height, texture_width, texture_channels = texture.shape

    # Redimensionar la textura si es más grande que la imagen
    if texture_height > height or texture_width > width:
        texture = cv2.resize(texture, (width, height))

    # Ajustar la intensidad de la textura
    texture *= intensity

    # Definir el kernel para superponer la textura
    overlay_kernel = SourceModule("""
        __global__ void overlay_texture(float *image, float *texture, int width, int height, int texture_width, int texture_height, int channels) {
            int x = threadIdx.x + blockIdx.x * blockDim.x;
            int y = threadIdx.y + blockIdx.y * blockDim.y;

            if (x < width && y < height) {
                int image_index = (y * width + x) * channels;
                int texture_index = (y * texture_width + x) * channels;
                for (int c = 0; c < channels; ++c) {
                    image[image_index + c] = fminf(image[image_index + c] + texture[texture_index + c], 255.0f);
                }
            }
        }
    """)

    # Obtener la función del kernel
    overlay_func = overlay_kernel.get_function("overlay_texture")

    # Transferir la imagen y la textura a la GPU
    image_gpu = cuda.mem_alloc(image.nbytes)
    texture_gpu = cuda.mem_alloc(texture.nbytes)
    cuda.memcpy_htod(image_gpu, image)
    cuda.memcpy_htod(texture_gpu, texture)

    # Definir las dimensiones del bloque y de la cuadrícula
    block = (16, 16, 1)
    grid = ((width + block[0] - 1) // block[0], (height + block[1] - 1) // block[1], 1)

    # Ejecutar el kernel en la GPU
    overlay_func(image_gpu, texture_gpu, np.int32(width), np.int32(height), np.int32(texture_width), np.int32(texture_height),
                 np.int32(channels), block=block, grid=grid)

    # Transferir la imagen de vuelta a la CPU
    cuda.memcpy_dtoh(image, image_gpu)

    # Convertir la imagen de vuelta a tipo uint8
    image = np.clip(image, 0, 255).astype(np.uint8)

    return image

# Cargar la imagen original
original_image = cv2.imread("rostro.jpg", cv2.IMREAD_COLOR)

# Cargar la textura a superponer
texture = cv2.imread("text.jpg", cv2.IMREAD_COLOR)

# Aplicar el filtro de superposición de texturas con intensidad modificada
result_image = overlay_texture(original_image, texture, intensity=1)

# Guardar la imagen resultante
cv2.imwrite("result_image.jpg", result_image)

# Mostrar la imagen resultante
cv2.imshow("Result Image", result_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [5]:
import numpy as np
import pycuda.autoinit
import pycuda.driver as cuda
from pycuda.compiler import SourceModule
import cv2

# Kernel para dibujar círculos y mantener el área dentro del círculo en color, y el área fuera en blanco y negro
kernel_code = """
__global__ void draw_circle_with_background(unsigned char *img, int width, int height, int cx, int cy, int radius) {
    int x = threadIdx.x + blockIdx.x * blockDim.x;
    int y = threadIdx.y + blockIdx.y * blockDim.y;
    if (x < width && y < height) {
        int dx = x - cx;
        int dy = y - cy;
        float distance = sqrtf(dx*dx + dy*dy);
        if (distance <= radius) {
            // Mantener el color dentro del círculo
            // No hacer nada, simplemente mantener el color original
        } else {
            // Convertir el área fuera del círculo a blanco y negro
            unsigned char gray_value = (unsigned char)(0.299*img[(y * width + x) * 3] + 0.587*img[(y * width + x) * 3 + 1] + 0.114*img[(y * width + x) * 3 + 2]);
            img[(y * width + x) * 3] = gray_value;
            img[(y * width + x) * 3 + 1] = gray_value;
            img[(y * width + x) * 3 + 2] = gray_value;
        }
    }
}
"""

def apply_circle_with_background_filter(image, radius, cx, cy):
    height, width, _ = image.shape

    # Copiar la imagen a la memoria de la GPU
    img_gpu = cuda.mem_alloc(image.nbytes)
    cuda.memcpy_htod(img_gpu, image)

    # Compilar el kernel
    mod = SourceModule(kernel_code)
    draw_circle_with_background_kernel = mod.get_function("draw_circle_with_background")

    # Definir las dimensiones del bloque y de la grilla
    block = (16, 16, 1)
    grid = ((width + block[0] - 1) // block[0], (height + block[1] - 1) // block[1])

    # Llamar al kernel
    draw_circle_with_background_kernel(img_gpu, np.int32(width), np.int32(height), np.int32(cx), np.int32(cy), np.int32(radius), block=block, grid=grid)

    # Copiar la imagen de vuelta a la CPU
    result = np.empty_like(image)
    cuda.memcpy_dtoh(result, img_gpu)

    # Guardar la imagen procesada en un archivo
    output_path = 'rostro_procesado.jpg'
    cv2.imwrite(output_path, result)
    print(f"Imagen procesada guardada en {output_path}")

# Ruta de la imagen
image_path = 'rostro.jpg'

# Cargar la imagen
image = cv2.imread(image_path)

# Comprobar si la imagen se ha cargado correctamente
if image is None:
    print("Error al cargar la imagen")
else:
    # Obtener dimensiones de la imagen
    height, width, _ = image.shape

    # Definir el radio del círculo
    radius = 100

    # Definir el centro del círculo en el centro de la imagen
    cx, cy = width // 2, height // 2

    # Aplicar el filtro de círculos con fondo en blanco y negro
    apply_circle_with_background_filter(image, radius, cx, cy)


Imagen procesada guardada en rostro_procesado.jpg


In [3]:
import numpy as np
import pycuda.autoinit
import pycuda.driver as cuda
from pycuda.compiler import SourceModule
from PIL import Image

# Kernel para dibujar círculos y mantener el área dentro del círculo en color, y el área fuera en blanco y negro
kernel_code = """
__global__ void draw_circle_with_background(unsigned char *img, int width, int height, int cx, int cy, int radius) {
    int x = threadIdx.x + blockIdx.x * blockDim.x;
    int y = threadIdx.y + blockIdx.y * blockDim.y;
    if (x < width && y < height) {
        int dx = x - cx;
        int dy = y - cy;
        float distance = sqrtf(dx*dx + dy*dy);
        if (distance <= radius) {
            // Mantener el color dentro del círculo
            // No hacer nada, simplemente mantener el color original
        } else {
            // Convertir el área fuera del círculo a blanco y negro
            unsigned char gray_value = (unsigned char)(0.299*img[(y * width + x) * 3] + 0.587*img[(y * width + x) * 3 + 1] + 0.114*img[(y * width + x) * 3 + 2]);
            img[(y * width + x) * 3] = gray_value;
            img[(y * width + x) * 3 + 1] = gray_value;
            img[(y * width + x) * 3 + 2] = gray_value;
        }
    }
}
"""

def apply_circle_with_background_filter(image, radius, cx, cy):
    height, width, _ = image.shape

    # Copiar la imagen a la memoria de la GPU
    img_gpu = cuda.mem_alloc(image.nbytes)
    cuda.memcpy_htod(img_gpu, image)

    # Compilar el kernel
    mod = SourceModule(kernel_code)
    draw_circle_with_background_kernel = mod.get_function("draw_circle_with_background")

    # Definir las dimensiones del bloque y de la grilla
    block = (16, 16, 1)
    grid = ((width + block[0] - 1) // block[0], (height + block[1] - 1) // block[1])

    # Llamar al kernel
    draw_circle_with_background_kernel(img_gpu, np.int32(width), np.int32(height), np.int32(cx), np.int32(cy), np.int32(radius), block=block, grid=grid)

    # Copiar la imagen de vuelta a la CPU
    result = np.empty_like(image)
    cuda.memcpy_dtoh(result, img_gpu)

    # Liberar la memoria de la GPU
    img_gpu.free()

    return result

# Ruta de la imagen
image_path = 'rostro.jpg'

# Cargar la imagen usando Pillow
image = Image.open(image_path)
image = image.convert('RGB')  # Asegurarse de que la imagen esté en formato RGB
image_np = np.array(image)

# Obtener dimensiones de la imagen
height, width, _ = image_np.shape

# Definir el radio del círculo
radius = 100

# Definir el centro del círculo en el centro de la imagen
cx, cy = width // 2, height // 2

# Aplicar el filtro de círculos con fondo en blanco y negro
filtered_image_np = apply_circle_with_background_filter(image_np, radius, cx, cy)

# Convertir la imagen resultante de nuevo a un objeto Image de Pillow
filtered_image = Image.fromarray(filtered_image_np)

# Guardar la imagen resultante
filtered_image.save('filtered_persona.png')




In [9]:
import numpy as np
import cv2
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

# Cargar la imagen
imagen_original = cv2.imread("rostro.jpg", cv2.IMREAD_COLOR)

# Convertir la imagen a escala de grises
imagen_gris = cv2.cvtColor(imagen_original, cv2.COLOR_BGR2GRAY)

# Crear un kernel para el desenfoque de caja
kernel_code = """
    __global__ void box_blur(int *input, int *output, int width, int height) {
        int x = threadIdx.x + blockIdx.x * blockDim.x;
        int y = threadIdx.y + blockIdx.y * blockDim.y;
        if (x < width && y < height) {
            int sum = 0;
            for (int i = -1; i <= 1; ++i) {
                for (int j = -1; j <= 1; ++j) {
                    int px = x + i;
                    int py = y + j;
                    if (px >= 0 && px < width && py >= 0 && py < height) {
                        sum += input[py * width + px];
                    }
                }
            }
            output[y * width + x] = sum / 9;
        }
    }
"""

mod = SourceModule(kernel_code)
box_blur_kernel = mod.get_function("box_blur")

# Copiar la imagen a la GPU
imagen_gris_gpu = cuda.to_device(imagen_gris)
imagen_resultado_gpu = cuda.mem_alloc(imagen_gris.nbytes)

# Definir el tamaño de bloque y cuadrícula
block = (16, 16, 1)
grid = (imagen_gris.shape[1] // block[0] + 1, imagen_gris.shape[0] // block[1] + 1)

# Aplicar el desenfoque de caja en la GPU
box_blur_kernel(imagen_gris_gpu, imagen_resultado_gpu, np.int32(imagen_gris.shape[1]), np.int32(imagen_gris.shape[0]), block=block, grid=grid)

# Copiar el resultado de vuelta a la CPU
imagen_resultado = np.empty_like(imagen_gris)
cuda.memcpy_dtoh(imagen_resultado, imagen_resultado_gpu)

# Guardar la imagen filtrada
cv2.imwrite("imagen_filtrada_pycuda.jpg", imagen_resultado)


True

In [1]:
import numpy as np
import cv2
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

# Cargar la imagen
imagen_original = cv2.imread("red-ius-dob-bosco.jpg", cv2.IMREAD_COLOR)

# Coordenadas del centro y radio del círculo
ancho, alto = imagen_original.shape[1], imagen_original.shape[0]
centro_x, centro_y = ancho // 2, alto // 2
radio = 100  # Ajusta el radio según tus necesidades

# Crear una máscara circular
mascara = np.zeros((alto, ancho), dtype=np.uint8)
cv2.circle(mascara, (centro_x, centro_y), radio, 1, -1)

# Copiar la imagen a la GPU
imagen_original_gpu = cuda.to_device(imagen_original)
mascara_gpu = cuda.to_device(mascara)
imagen_resultado_gpu = cuda.mem_alloc(imagen_original.nbytes)

# Definir el kernel para aplicar la máscara
kernel_code = """
    __global__ void apply_mask(int *input, int *mask, int *output, int width, int height) {
        int x = threadIdx.x + blockIdx.x * blockDim.x;
        int y = threadIdx.y + blockIdx.y * blockDim.y;
        if (x < width && y < height) {
            int index = y * width + x;
            if (mask[index] == 1) {
                output[index] = input[index];
            } else {
                int gris = 0.2989 * input[index] + 0.587 * input[index + width] + 0.114 * input[index + 2 * width];
                output[index] = gris;
            }
        }
    }
"""

mod = SourceModule(kernel_code)
apply_mask_kernel = mod.get_function("apply_mask")

# Definir el tamaño de bloque y cuadrícula
block = (16, 16, 1)
grid = (ancho // block[0] + 1, alto // block[1] + 1)

# Aplicar la máscara en la GPU
apply_mask_kernel(imagen_original_gpu, mascara_gpu, imagen_resultado_gpu, np.int32(ancho), np.int32(alto), block=block, grid=grid)

# Copiar el resultado de vuelta a la CPU
imagen_resultado = np.empty_like(imagen_original)
cuda.memcpy_dtoh(imagen_resultado, imagen_resultado_gpu)

# Guardar la imagen filtrada
cv2.imwrite("imagen_filtrada_pycuda1.jpg", imagen_resultado)


True

In [1]:
!pip install qrcode

  Obtaining dependency information for qrcode from https://files.pythonhosted.org/packages/24/79/aaf0c1c7214f2632badb2771d770b1500d3d7cbdf2590ae62e721ec50584/qrcode-7.4.2-py3-none-any.whl.metadata
  Obtaining dependency information for pypng from https://files.pythonhosted.org/packages/3e/b9/3766cc361d93edb2ce81e2e1f87dd98f314d7d513877a342d31b30741680/pypng-0.20220715.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/46.2 kB ? eta -:--:--
   ----------------------------------- ---- 41.0/46.2 kB 991.0 kB/s eta 0:00:01
   ---------------------------------------- 46.2/46.2 kB 765.0 kB/s eta 0:00:00
   ---------------------------------------- 0.0/58.1 kB ? eta -:--:--
   ---------------------------------------- 58.1/58.1 kB 3.2 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import qrcode

# URL de descarga del archivo
url_descarga = "https://drive.google.com/file/d/1mGmyIb4QG2nJZbruvQEM5cr4WwftnetS/view?usp=sharing"  # Cambia esto por la URL real del archivo

# Genera el código QR con la URL de descarga
qr = qrcode.make(url_descarga)

# Guarda el código QR en un archivo de imagen (por ejemplo, "archivo_qr.png")
qr.save("archivo_qr.png")

print("¡Código QR generado y guardado en archivo_qr.png!")


¡Código QR generado y guardado en archivo_qr.png!


In [1]:
from PIL import Image

# Abre la imagen en color
imagen = Image.open("C:/Users/lcres/Downloads/yessy_web/yessy_web/img/logomomish.png")

# Convierte la imagen a blanco y negro
imagen_bn = imagen.convert("1")

# Guarda la imagen resultante
imagen_bn.save("imagen_blanco_negro.png")


In [2]:
from PIL import Image

# Abre la imagen
imagen = Image.open("C:/Users/lcres/Downloads/yessy_web/yessy_web/img/logomomish.png")

# Convierte los píxeles negros a blancos
for x in range(imagen.width):
    for y in range(imagen.height):
        r, g, b, a = imagen.getpixel((x, y))
        if r == 0 and g == 0 and b == 0:
            imagen.putpixel((x, y), (255, 255, 255, a))

# Guarda la imagen resultante
imagen.save("imagen_con_lineas_blancas.png")
